In [1]:
import pickle
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# import final dataframe
cwd = os.getcwd()
final_df = pickle.load(open( cwd+"\\data\\final_df.pkl", "rb" ) )

In [4]:
len(final_df)

1612557